# 朴素贝叶斯中的拉普拉斯修正（Laplace Smoothing）

在使用朴素贝叶斯（Naive Bayes）分类器进行文本分类、垃圾邮件识别等任务时，经常会遇到**概率为 0 的问题**。这时，就需要用到**拉普拉斯修正**（也叫**加一平滑**）来解决这一问题。

---

## 📌 为什么需要拉普拉斯修正？

朴素贝叶斯模型中，条件概率通常按如下公式计算：

```math
P(x_i | C_k) = \frac{N_{ik} + 1}{N_k + V}
```
其中：

- `V` 表示词汇表（所有可能出现的词）的大小
- 分子加 1，表示“每个词至少出现 1 次”
- 分母加 `V`，保证概率归一化

这被称为 **加一平滑（Add-one Smoothing）**。

更通用的做法是加任意一个正数 α（称为 **Lidstone 平滑**）：

```math
P(x_i | C_k) = \frac{N_{ik} + α}{N_k + αV}
```
当 α = 1 时，就是拉普拉斯修正。

---

## 🧠 示例

假设我们的词汇表是：`["apple", "banana", "cherry"]`，共有 3 个词，即 `V = 3`。

在类别 `"fruit"` 中，统计如下：

- `"apple"` 出现 3 次
- `"banana"` 出现 2 次
- `"cherry"` 没出现（次数为 0）
- 总词数为 5，即 `N(c) = 5`

如果不使用拉普拉斯修正，`P("cherry" | "fruit") = 0 / 5 = 0`。

使用拉普拉斯修正后：

```python
P("cherry" | "fruit") = (0 + 1) / (5 + 3) = 1 / 8 = 0.125
```
这样就避免了 0 概率的问题。

---

## ⚙ 应用意义

- ✅ **避免零概率**：保证每个特征在每个类别下都有非零概率。
- 🛡️ **提高鲁棒性**：增强模型对未见过特征的适应能力。
- 📊 **适用于稀疏数据场景**：尤其在文本分类、高维特征下效果显著。

---

## 📚 延伸阅读

- 加权平滑（Lidstone Smoothing）
- Kneser-Ney 平滑（用于语言模型）
- Scikit-learn 中 `MultinomialNB` 的 `alpha` 参数控制平滑强度

In [ ]:
def Train(train_set,train_labels):
    m = train_set.shape[0]#样本数
    n = train_set.shape[1]#特征数
    prior_probability = {}#存储先验概率，key上类别值，value为概率
    conditional_probability = {}
    labels = set(train_labels)
    for label in labels:
        # prior_probability[label] = len(train_labels[train_labels == label])修正前
        prior_probability[label] = len(train_labels[train_labels == label])+1
    for i in range(m):
        for j in range(n):
            key = str(train_labels[i])+','+str(j)+','+str(train_set[i][j])
            if key in conditional_probability:
                conditional_probability[key] += 1
            else:
                conditional_probability[key] = 1
    conditional_probability_final = {}
    for key in conditional_probability:
        # label = key.split(',')[0]
        # conditional_probability_final[key] = conditional_probability[key]/prior_probability[int(label)]
        conditional_probability[key] = conditional_probability[key]+1
        key1 = int(key.split(','))
        Ni = len(set(train_set[:,key1]))
        label = key.split(',')[0]
        conditional_probability_final[key] = conditional_probability[key]/prior_probability[int(label)]+Ni
    for label in labels:
        prior_probability[label] = prior_probability[label]/m
    return prior_probability,conditional_probability_final, labels
